### Eager execution basics

In [1]:
import tensorflow as tf

In [2]:
tf.enable_eager_execution()

In [3]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64('hello world'))

# operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


In [4]:
# each tensor has a shape and a datatype
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


In [5]:
# Numpy compatibility
import numpy as np

In [6]:
ndarray = np.ones([3, 3])
print('Tensorflow op convert numpy arrays to tensors automatically')
tensor = tf.multiply(ndarray, 42)
print(tensor)

print('Add numpy op convert tensors to numpy arrays automatically')
print(np.add(tensor, 1))

print('the .numpy() method explicitly converts a tensor to a numpy array')
print(tensor.numpy())

Tensorflow op convert numpy arrays to tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
Add numpy op convert tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
the .numpy() method explicitly converts a tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [7]:
# GPU acceleration
x = tf.random_uniform([3, 3])
print('Is there a GPU available: ', tf.test.is_gpu_available())
print('Is the tensor on GPU #0: ', x.device.endswith('GPU:0'))

Is there a GPU available:  True
Is the tensor on GPU #0:  True


In [9]:
# explicit device placement
def time_matmul(x):
    %timeit tf.matmul(x, x)
    
# force execution on GPU
print('On CPU:')
with tf.device('CPU:0'):
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith('CPU:0')
    time_matmul(x)
    
# force execution on GPU #0 if available
if tf.test.is_gpu_available():
    print('On GPU:')
    with tf.device('GPU:0'):
        x = tf.random_uniform([1000, 1000])
        assert x.device.endswith('GPU:0')
        time_matmul(x)

On CPU:
11.7 ms ± 287 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
On GPU:
618 µs ± 6.31 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [10]:
# Datasets
# create a source Dataset
ds_tensor = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write('''
    Line 1
    Line 2
    Line 3
    ''')
    
ds_file = tf.data.TextLineDataset(filename)

In [11]:
# apply trainsformations
ds_tensor = ds_tensor.map(tf.square).shuffle(2).batch(2)
ds_file = ds_file.batch(2)

In [12]:
# Iterate
print('elements of ds_tensor:')
for x in ds_tensor:
    print(x)
    
print('\nelements in ds_file:')
for x in ds_file:
    print(x)

elements of ds_tensor:
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 16], shape=(2,), dtype=int32)
tf.Tensor([25 36], shape=(2,), dtype=int32)

elements in ds_file:
tf.Tensor([b'' b'    Line 1'], shape=(2,), dtype=string)
tf.Tensor([b'    Line 2' b'    Line 3'], shape=(2,), dtype=string)
tf.Tensor([b'    '], shape=(1,), dtype=string)
